In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./dataset_1.csv")
df.columns = df.columns.to_series().apply(lambda x: x.strip())

In [3]:
df.head(24)

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,NaN,8,6,0,2,2,14
2,NaN,9,8,3,0,0,14
3,NaN,10,9,2,0,1,14
4,NaN,11,11,1,4,4,11
5,NaN,12,12,0,2,2,11
6,NaN,13,9,1,0,0,9
7,NaN,14,12,1,0,0,9
8,NaN,15,11,2,1,2,7
9,NaN,16,11,2,3,4,6


In [4]:
df.fillna(method='ffill', inplace=True)

In [5]:
df.head(24)

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers
0,10-Sep-12,7,5,0,2,2,9
1,10-Sep-12,8,6,0,2,2,14
2,10-Sep-12,9,8,3,0,0,14
3,10-Sep-12,10,9,2,0,1,14
4,10-Sep-12,11,11,1,4,4,11
5,10-Sep-12,12,12,0,2,2,11
6,10-Sep-12,13,9,1,0,0,9
7,10-Sep-12,14,12,1,0,0,9
8,10-Sep-12,15,11,2,1,2,7
9,10-Sep-12,16,11,2,3,4,6


## Q1
Which date had the most completed trips during the two week period?

In [6]:
trips_by_day = df.groupby('Date').sum()[['Completed Trips']]
trips_by_day

,Completed Trips
Date,
10-Sep-12,26
11-Sep-12,40
12-Sep-12,91
13-Sep-12,45
14-Sep-12,108
15-Sep-12,199
16-Sep-12,93
17-Sep-12,57
18-Sep-12,42


In [7]:
trips_by_day['Completed Trips'].nlargest(1)

Date
22-Sep-12    248
Name: Completed Trips, dtype: int64

## A1: 22-Sep-12 has the largest number of completed trips during the two-week period with 248

## Q2
What was the highest number of completed trips within a 24 hour period?


In [8]:
def create_timestamp(d, t):
    return pd.to_datetime(f"{d} {t}:00")

In [9]:
# df['Timestamp'] = df.apply(lambda row: create_timestamp(row['Date'], row['Time (Local)']), axis=1)

df['Timestamp'] = df['Date'] + " " + df['Time (Local)'].astype(str) + ":00"
df['Timestamp']=pd.to_datetime(df['Timestamp'])



In [10]:
df['Trips in the last 24 Hour'] = df['Completed Trips'].rolling(24).sum()

In [11]:
df.nlargest(columns='Trips in the last 24 Hour',n=1)

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Timestamp,Trips in the last 24 Hour
297,22-Sep-12,16,58,17,15,18,16,2012-09-22 16:00:00,278.0


## A2: the highest number of completed trips within a 24 hour period is 278, which happened in the 24 hour period prior to 2012-09-22 17:00:00 (since the time stamp is at the start of the hour).


## Q3 Which hour of the day had the most requests during the two week period?

In [12]:
requests_by_hour = df.groupby('Time (Local)').mean()[['Requests']]
requests_by_hour.head(3)

,Requests
Time (Local),
0,10.142857
1,6.857143
2,7.142857


In [13]:
requests_by_hour.nlargest(columns='Requests',n=1)

,Requests
Time (Local),
23,13.142857


## A3: 11pm had the most requests

## Q4 What percentages of all zeroes during the two week period occurred on weekend (Friday at 5 pm to Sunday at 3 am)? Tip: The local time value is the start of the hour (e.g. 15 is the hour from 3:00pm - 4:00pm)

In [14]:
df.head()

,Date,Time (Local),Eyeballs,Zeroes,Completed Trips,Requests,Unique Drivers,Timestamp,Trips in the last 24 Hour
0,10-Sep-12,7,5,0,2,2,9,2012-09-10 07:00:00,NaN
1,10-Sep-12,8,6,0,2,2,14,2012-09-10 08:00:00,NaN
2,10-Sep-12,9,8,3,0,0,14,2012-09-10 09:00:00,NaN
3,10-Sep-12,10,9,2,0,1,14,2012-09-10 10:00:00,NaN
4,10-Sep-12,11,11,1,4,4,11,2012-09-10 11:00:00,NaN


In [15]:
total_zeros = df.Zeroes.sum()
total_zeros

1429

In [16]:
df['Day of Week'] = pd.to_datetime(df['Date']).dt.dayofweek

In [17]:
weekend_zeros = df[
    ((df['Day of Week']==4) & (df['Time (Local)']>=17)) |
    (df['Day of Week']==5) |
    ((df['Day of Week']==6) & (df['Time (Local)']<3))]['Zeroes'].sum()

In [18]:
weekend_zeros/total_zeros

0.44856543037088875

## Q5
What is the weighted average ratio of completed trips per driver during the two week period?

In [19]:
df['completed_trip_ratio_per_driver'] = df['Completed Trips'] / df['Unique Drivers']
df_nonan = df[(df['Unique Drivers'] > 0)]

In [24]:
weighted_average_ratio = (df_nonan['completed_trip_ratio_per_driver'] * (df_nonan['Completed Trips'] / df_nonan['Completed Trips'].sum())).mean()
weighted_average_ratio

0.002788813372084982

In [21]:
not_weighted = df_nonan['completed_trip_ratio_per_driver'].mean()
not_weighted

0.4743397876937869